In [45]:
from pyteaser import Summarize, SummarizeUrl
from pprint import pprint
import pandas as pd
import numpy as np
import tagui as t
import sqlite3
from sqlite3 import Error
from datetime import datetime

In [67]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("DB Connection setup")
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn
        else:
            print("DB Connection failed")


if __name__ == '__main__':
    conn = create_connection("./db/news.db")

DB Connection setup


In [10]:
def getSiteByName(name):
    cur = conn.cursor()
    cur.execute("SELECT * FROM site where name=?",(name,))
    rows = cur.fetchall()
    return rows

In [51]:
site = getSiteByName("StraitsTimes")
url = site[0][2]
site_id = site[0][0]

In [21]:
t.init(visual_automation = True, chrome_browser = True)
t.url(url)

True

In [22]:
number_bb = t.count('(//div[contains(@data-vr-zone, "Top Stories")]//span[contains(@class, "story-headline")])')

df_bb = pd.DataFrame(index=range(0,number_bb-2), columns = ['Sno', 'Title', 'URL', 'Summary'])


for n in range(0, number_bb-2):
    title=t.read('//div[contains(@data-vr-zone, "Top Stories {}")]//span[contains(@class, "story-headline")]'.format(n))
    URL_b=t.read('//div[contains(@data-vr-zone, "Top Stories {}")]//span[contains(@class, "story-headline")]//@href'.format(n))
    URL = "https://www.straitstimes.com/" + str(URL_b) 
    summaries = SummarizeUrl(URL)
    
    df_bb.iloc[n, 0] = n
    df_bb.iloc[n, 1] = title
    df_bb.iloc[n, 2] = URL
    df_bb.iloc[n, 3] = summaries
    


c:\users\superhell\.conda\envs\ca1\lib\site-packages\BeautifulSoup.py:114: UserWarning: You are using a very old release of Beautiful Soup, last updated in 2011. If you installed the 'beautifulsoup' package through pip, you should know the 'beautifulsoup' package name is about to be reclaimed by a more recent version of Beautiful Soup which is incompatible with this version.

This will happen at some point after January 1, 2021.

If you just started this project, this is easy to fix. Install the 'beautifulsoup4' package instead of 'beautifulsoup' and start using Beautiful Soup 4.

If this is an existing project that depends on Beautiful Soup 3, the project maintainer (potentially you) needs to start the process of migrating to Beautiful Soup 4. This should be a relatively easy part of the Python 3 migration.

  """)


,Sno,Title,URL,Summary
0,0,S'pore economy to open up 'step by step' after...,https://www.straitstimes.com//singapore/spore-...,[The coronavirus pandemic will leave its mark ...
1,1,Migrant worker hospitalised in Sengkang Hospit...,https://www.straitstimes.com//singapore/migran...,"[""May Day is a day of celebration for all work..."
2,2,58-year-old S'porean woman dies from coronavir...,https://www.straitstimes.com//singapore/health...,"[MOH also announced 12 new virus clusters, inc..."
3,3,Infection numbers in dorms could take a few we...,https://www.straitstimes.com//singapore/infect...,"[For now, Singapore is still seeing ""very larg..."
4,4,Online jazz concert Jass@Home chases Covid-19 ...,https://www.straitstimes.com//singapore/online...,[SINGAPORE - A jazz concert that aired on Face...
5,5,Duke-NUS scientists develop speedy test for an...,https://www.straitstimes.com//singapore/health...,[He added that tests that can detect neutralis...
6,6,Banks to allow temporary halt in payments for ...,https://www.straitstimes.com//business/banking...,[OCBC Bank said on Thursday that it had starte...
7,7,McDonald's extends closure of S'pore restauran...,https://www.straitstimes.com//singapore/mcdona...,[SINGAPORE - Singaporeans looking forward to g...
8,8,Changi Airport T2 proceeds with plans to suspe...,https://www.straitstimes.com//singapore/transp...,"[Changi Airport Group (CAG) said: ""The suspens..."


In [70]:
def insertNews(conn,news):    
    sql = ''' INSERT INTO news(site_id,create_date,title,url,summary)
              VALUES(?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, news)
    return cur.lastrowid

In [77]:
now = datetime.now()
# dd/mm/YY
dt_string = now.strftime("%d/%m/%Y")

for index, row in df_bb.iterrows():
    title = row['Title']
    url = row['URL']
    summary = str(row['Summary'])
    
    news = (site_id,dt_string,title,url,summary)
    news_id = insertNews(conn,news)
    print("News %d has been created" % news_id)
    
conn.commit()

News 1 has been created
News 2 has been created
News 3 has been created
News 4 has been created
News 5 has been created
News 6 has been created
News 7 has been created
News 8 has been created
News 9 has been created


In [78]:
conn.close()